# Scrape NBA Player Game Logs from ESPN with lxml

## Building the Scraper
### Import modules

In [153]:
import csv
import time
import string
import datetime
import requests
import lxml.html as lh
import json
import numpy as np
import pandas as pd

In [154]:
def parse_table(table):
    tr_elements = table.xpath('./tbody//tr')
    for tr_element in tr_elements:
        td_elements = tr_element.xpath('./td')
        if td_elements:
            for td_element in td_elements:
                print('td_element: ', td_element.text_content())

### Load RPM data and save to new DataFrame

In [155]:
# Load rpm data to extract game log urls
rpm = pd.read_csv('nba_rpm_data_2018.csv')
rpm = rpm.drop(columns='Unnamed: 0')
rpm.head()

,RK,NAME,POS,TEAM,GP,MPG,ORPM,DRPM,RPM,WINS,GAME LOG PAGE
0,1,Paul George,SF,OKC,77,36.9,4.48,3.08,7.56,19.73,http://www.espn.com/nba/player/gamelog/_/id/42...
1,2,James Harden,PG,HOU,78,36.8,7.41,0.00,7.41,18.53,http://www.espn.com/nba/player/gamelog/_/id/39...
2,3,Stephen Curry,PG,GS,69,33.8,5.92,0.81,6.73,15.07,http://www.espn.com/nba/player/gamelog/_/id/39...
3,4,Giannis Antetokounmpo,PF,MIL,72,32.8,3.12,3.39,6.51,14.94,http://www.espn.com/nba/player/gamelog/_/id/30...
4,5,Joel Embiid,C,PHI,64,33.7,2.70,3.71,6.41,12.91,http://www.espn.com/nba/player/gamelog/_/id/30...


### Save urls to list

In [156]:
urls = list(rpm['GAME LOG PAGE'])
print('urls[:5]: ', urls[:5])

urls[:5]:  ['http://www.espn.com/nba/player/gamelog/_/id/4251/paul-george', 'http://www.espn.com/nba/player/gamelog/_/id/3992/james-harden', 'http://www.espn.com/nba/player/gamelog/_/id/3975/stephen-curry', 'http://www.espn.com/nba/player/gamelog/_/id/3032977/giannis-antetokounmpo', 'http://www.espn.com/nba/player/gamelog/_/id/3059318/joel-embiid']


### Loop through the list and scrape each url

In [ ]:
for url in urls:
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    div = doc.xpath('//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/div[1]/div/div[3]')[0]
    tables = div.xpath('//table[@class=\'Table2__table-scroller Table2__right-aligned Table2__table\']')
    tables = tables[2:-3]
    
    # ...

In [ ]:
url = 'http://www.espn.com/nba/player/gamelog/_/id/4251/paul-george'

page = requests.get(url)

doc = lh.fromstring(page.content)

div = doc.xpath('//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/div[1]/div/div[3]')

# Convert list to single item
div = div[0]

print('div: ', div)

In [ ]:
# Now get the divs that contain the stats tables for each month
divs = div.xpath('//div[@class=\'mb5\']')
print('divs: ', divs)

In [ ]:
tables = div.xpath('//table[@class=\'Table2__table-scroller Table2__right-aligned Table2__table\']')
tables = tables[2:-3]
print('tables: ', tables)

### Loop through each game log table and extract its `<td>` elements

In [ ]:
for url in urls:

    # ...
    
    # Create empty dataframe with columns
    df = pd.DataFrame(columns=['Date', 'Day of Week', 'Opp', 'Location', 'Result', 'Score', 'Min', 'FG', 'FG%', '3PT', '3PT%', 'FT', 'FT%', 'REB', 'AST', 'BLK', 'STL', 'PF', 'TO', 'PTS', 'FPTS'])
    months = ['october', 'november', 'december', 'january', 'february', 'march', 'april']
    game_log = []

    for table in tables:
        # Get <tr> elements
        tr_elements = table.xpath('./tbody//tr')
        for tr_element in tr_elements:
            # Get <td> elements
            td_elements = tr_element.xpath('./td[@class=\'Table2__td\']')
            
            # ...


### Parse each row of the table
* Extract the date
* Determine if game is at home or away
* Determine if game result was a win or loss
* Calculate total Fanduel points scored
* Add game to game log DataFrame

In [ ]:
for url in urls:

    # ...

    for table in tables:
    
        # ...

            if td_elements:
                # Ignore average stats row
                if td_elements[0].text_content() in months:
                    break
                count = 0
                game = []
                for td_element in td_elements:
                    # Get date
                    if count == 0:
                        game.append(td_element.text_content().split(' ')[1])
                        game.append(td_element.text_content().split(' ')[0])
                    # Determine if home/road
                    elif count == 1:
                        if 'vs' in td_element.text_content():
                            game.append(td_element.text_content().replace('vs', ''))
                            game.append('Home')
                        elif '@' in td_element.text_content():
                            game.append(td_element.text_content().replace('@', ''))
                            game.append('Away')
                    # Determine if win/loss
                    elif count == 2:
                        if 'W' in td_element.text_content():
                            game.append('W')
                            game.append(td_element.text_content().replace('W', ''))
                        elif 'L' in td_element.text_content():
                            game.append('L')
                            game.append(td_element.text_content().replace('L', ''))
                    elif count > 2:
                        game.append(td_element.text_content())
                    count += 1
                    
                    # ...

### Calculate Fanduel points and add row to game log DataFrame

In [ ]:
for url in urls:

    # ...

    for table in tables:
    
        # ...

            if td_elements:

                # ...
                
                for td_element in td_elements:

                    # ...
                    
                # Calculate total Fanduel points scored
                game.append(1.2*int(game[13])+1.5*int(game[14])+3*int(game[15])+3*int(game[16])+int(game[19])-int(game[18]))

                # Add to game log
                df.loc[len(df)] = game

## Our Complete Scraper

In [163]:
import csv
import time
import string
import datetime
import requests
import lxml.html as lh
import json
import numpy as np
import pandas as pd

# Load rpm data to extract game log urls
rpm = pd.read_csv('nba_rpm_data_2018.csv')
rpm = rpm.drop(columns='Unnamed: 0')
rpm.head()

urls = list(rpm['GAME LOG PAGE'])

for url in urls:
    # Create empty dataframe with columns
    df = pd.DataFrame(columns=['Date', 'Day of Week', 'Opp', 'Location', 'Result', 'Score', 'Min', 'FG', 'FG%', '3PT', '3PT%', 'FT', 'FT%', 'REB', 'AST', 'BLK', 'STL', 'PF', 'TO', 'PTS', 'FPTS'])
    months = ['october', 'november', 'december', 'january', 'february', 'march', 'april']
    game_log = []

    for table in tables:
        # Get <tr> elements
        tr_elements = table.xpath('./tbody//tr')
        for tr_element in tr_elements:
            # Get <td> elements
            td_elements = tr_element.xpath('./td[@class=\'Table2__td\']')
            if td_elements:
                # Ignore average stats row
                if td_elements[0].text_content() in months:
                    break
                count = 0
                game = [] # to store data from a single game
                for td_element in td_elements:
                    print('td_element: ', td_element.text_content())
                    print('count: ', count)                
                    # Get date
                    if count == 0:
                        print('count = 0\n')
                        print('Date: {}\n'.format(td_element.text_content().split(' ')[1]))
                        print('Day of Week: {}\n'.format(td_element.text_content().split(' ')[0]))
                        game.append(td_element.text_content().split(' ')[1])
                        game.append(td_element.text_content().split(' ')[0])
                    # Determine if home/road
                    elif count == 1:
                        if 'vs' in td_element.text_content():
                            game.append(td_element.text_content().replace('vs', ''))
                            game.append('Home')
                        elif '@' in td_element.text_content():
                            game.append(td_element.text_content().replace('@', ''))
                            game.append('Away')
                    # Determine if win/loss
                    elif count == 2:
                        if 'W' in td_element.text_content():
                            game.append('W')
                            game.append(td_element.text_content().replace('W', ''))
                        elif 'L' in td_element.text_content():
                            game.append('L')
                            game.append(td_element.text_content().replace('L', ''))
                    elif count > 2:
                        game.append(td_element.text_content())
                    count += 1
                # Calculate total Fanduel points scored
                game.append(1.2*int(game[13])+1.5*int(game[14])+3*int(game[15])+3*int(game[16])+int(game[19])-int(game[18]))

                # Add to game log
                df.loc[len(df)] = game
    break

    df.head()

td_element:  Sun 4/7
count:  0
count = 0

Date: 4/7

Day of Week: Sun

td_element:  @BOS
count:  1
td_element:  W116-108
count:  2
td_element:  34
count:  3
td_element:  12-22
count:  4
td_element:  54.5
count:  5
td_element:  1-4
count:  6
td_element:  25.0
count:  7
td_element:  0-0
count:  8
td_element:  0.0
count:  9
td_element:  12
count:  10
td_element:  4
count:  11
td_element:  2
count:  12
td_element:  1
count:  13
td_element:  3
count:  14
td_element:  3
count:  15
td_element:  25
count:  16
td_element:  Fri 4/5
count:  0
count = 0

Date: 4/5

Day of Week: Fri

td_element:  vsATL
count:  1
td_element:  W149-113
count:  2
td_element:  30
count:  3
td_element:  9-13
count:  4
td_element:  69.2
count:  5
td_element:  2-4
count:  6
td_element:  50.0
count:  7
td_element:  5-6
count:  8
td_element:  83.3
count:  9
td_element:  11
count:  10
td_element:  4
count:  11
td_element:  0
count:  12
td_element:  1
count:  13
td_element:  3
count:  14
td_element:  2
count:  15
td_element: 

count:  7
td_element:  4-5
count:  8
td_element:  80.0
count:  9
td_element:  17
count:  10
td_element:  3
count:  11
td_element:  0
count:  12
td_element:  2
count:  13
td_element:  1
count:  14
td_element:  0
count:  15
td_element:  25
count:  16
td_element:  Sun 2/10
count:  0
count = 0

Date: 2/10

Day of Week: Sun

td_element:  @ATL
count:  1
td_element:  W124-108
count:  2
td_element:  29
count:  3
td_element:  8-13
count:  4
td_element:  61.5
count:  5
td_element:  1-2
count:  6
td_element:  50.0
count:  7
td_element:  2-2
count:  8
td_element:  100.0
count:  9
td_element:  12
count:  10
td_element:  3
count:  11
td_element:  1
count:  12
td_element:  0
count:  13
td_element:  2
count:  14
td_element:  1
count:  15
td_element:  19
count:  16
td_element:  Sat 2/9
count:  0
count = 0

Date: 2/9

Day of Week: Sat

td_element:  @MIL
count:  1
td_element:  W103-83
count:  2
td_element:  26
count:  3
td_element:  7-16
count:  4
td_element:  43.8
count:  5
td_element:  1-4
count:  6
td

count:  13
td_element:  1
count:  14
td_element:  1
count:  15
td_element:  18
count:  16
td_element:  Sun 1/6
count:  0
count = 0

Date: 1/6

Day of Week: Sun

td_element:  @LAC
count:  1
td_element:  L106-96
count:  2
td_element:  32
count:  3
td_element:  7-17
count:  4
td_element:  41.2
count:  5
td_element:  2-5
count:  6
td_element:  40.0
count:  7
td_element:  0-0
count:  8
td_element:  0.0
count:  9
td_element:  24
count:  10
td_element:  8
count:  11
td_element:  2
count:  12
td_element:  0
count:  13
td_element:  1
count:  14
td_element:  3
count:  15
td_element:  16
count:  16
td_element:  Fri 1/4
count:  0
count = 0

Date: 1/4

Day of Week: Fri

td_element:  @MIN
count:  1
td_element:  L120-103
count:  2
td_element:  27
count:  3
td_element:  10-16
count:  4
td_element:  62.5
count:  5
td_element:  1-4
count:  6
td_element:  25.0
count:  7
td_element:  1-1
count:  8
td_element:  100.0
count:  9
td_element:  7
count:  10
td_element:  3
count:  11
td_element:  1
count:  12
td

count:  15
td_element:  20
count:  16
td_element:  Mon 11/26
count:  0
count = 0

Date: 11/26

Day of Week: Mon

td_element:  @GS
count:  1
td_element:  L116-110
count:  2
td_element:  33
count:  3
td_element:  12-21
count:  4
td_element:  57.1
count:  5
td_element:  1-4
count:  6
td_element:  25.0
count:  7
td_element:  5-6
count:  8
td_element:  83.3
count:  9
td_element:  12
count:  10
td_element:  6
count:  11
td_element:  0
count:  12
td_element:  1
count:  13
td_element:  4
count:  14
td_element:  1
count:  15
td_element:  30
count:  16
td_element:  Sun 11/25
count:  0
count = 0

Date: 11/25

Day of Week: Sun

td_element:  @LAL
count:  1
td_element:  W108-104
count:  2
td_element:  36
count:  3
td_element:  10-20
count:  4
td_element:  50.0
count:  5
td_element:  3-8
count:  6
td_element:  37.5
count:  7
td_element:  8-8
count:  8
td_element:  100.0
count:  9
td_element:  15
count:  10
td_element:  7
count:  11
td_element:  3
count:  12
td_element:  2
count:  13
td_element:  2
co

## Data Engineering

### Remove All-Star exhibition from game log

In [164]:
# Remove All-Star game from gamelog
df = df[df.Opp != 'LEB*']
df = df[df.Opp != 'GIA*']

### Add year to 'Date' column

In [165]:
# Data engineering
df['Date'] = df['Date'].apply(lambda x: x + '/19' if int(x.split('/')[0]) > 0 and int(x.split('/')[0]) <= 4 else x + '/18')
df.head()

,Date,Day of Week,Opp,Location,Result,Score,Min,FG,FG%,3PT,...,FT,FT%,REB,AST,BLK,STL,PF,TO,PTS,FPTS
0,4/7/19,Sun,BOS,Away,W,116-108,34,12-22,54.5,1-4,...,0-0,0.0,12,4,2,1,3,3,25,51.4
1,4/5/19,Fri,ATL,Home,W,149-113,30,9-13,69.2,2-4,...,5-6,83.3,11,4,0,1,3,2,25,45.2
2,4/3/19,Wed,NY,Home,W,114-100,33,11-21,52.4,1-3,...,6-8,75.0,13,3,1,0,4,3,29,49.1
3,4/1/19,Mon,TOR,Away,L,121-109,26,5-14,35.7,2-3,...,1-2,50.0,13,3,0,0,2,2,13,31.1
4,3/30/19,Sat,IND,Away,W,121-116,32,8-18,44.4,0-3,...,3-4,75.0,2,2,0,2,2,3,19,27.4


### Add 'Days Rest' column

In [166]:
# Insert column with empty cells
df.insert(2, 'Days Rest', '')

# Convert date to datetime format
df['Date'] = df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%y'))

# Calculate days rest since previous game
df['Days Rest'] = (df['Date'] - df['Date'].shift(-1)).dt.days

df.head()

,Date,Day of Week,Days Rest,Opp,Location,Result,Score,Min,FG,FG%,...,FT,FT%,REB,AST,BLK,STL,PF,TO,PTS,FPTS
0,2019-04-07,Sun,2.0,BOS,Away,W,116-108,34,12-22,54.5,...,0-0,0.0,12,4,2,1,3,3,25,51.4
1,2019-04-05,Fri,2.0,ATL,Home,W,149-113,30,9-13,69.2,...,5-6,83.3,11,4,0,1,3,2,25,45.2
2,2019-04-03,Wed,2.0,NY,Home,W,114-100,33,11-21,52.4,...,6-8,75.0,13,3,1,0,4,3,29,49.1
3,2019-04-01,Mon,2.0,TOR,Away,L,121-109,26,5-14,35.7,...,1-2,50.0,13,3,0,0,2,2,13,31.1
4,2019-03-30,Sat,2.0,IND,Away,W,121-116,32,8-18,44.4,...,3-4,75.0,2,2,0,2,2,3,19,27.4


### Do rolling calculations

In [167]:
# Do rolling calculations
# Rolling calculation template
# df[''] = df[''].iloc[::-1].rolling().mean()

df['FPTS LAST 3'] = df['FPTS'].iloc[::-1].rolling(3).mean()
df['FPTS LAST 5'] = df['FPTS'].iloc[::-1].rolling(5).mean()
df['FPTS LAST 10'] = df['FPTS'].iloc[::-1].rolling(10).mean()

df.head(100)

,Date,Day of Week,Days Rest,Opp,Location,Result,Score,Min,FG,FG%,...,AST,BLK,STL,PF,TO,PTS,FPTS,FPTS LAST 3,FPTS LAST 5,FPTS LAST 10
0,2019-04-07,Sun,2.0,BOS,Away,W,116-108,34,12-22,54.5,...,4,2,1,3,3,25,51.4,48.566667,40.84,42.97
1,2019-04-05,Fri,2.0,ATL,Home,W,149-113,30,9-13,69.2,...,4,0,1,3,2,25,45.2,41.800000,36.84,44.08
2,2019-04-03,Wed,2.0,NY,Home,W,114-100,33,11-21,52.4,...,3,1,0,4,3,29,49.1,35.866667,38.74,43.58
3,2019-04-01,Mon,2.0,TOR,Away,L,121-109,26,5-14,35.7,...,3,0,0,2,2,13,31.1,29.966667,38.36,42.95
4,2019-03-30,Sat,2.0,IND,Away,W,121-116,32,8-18,44.4,...,2,0,2,2,3,19,27.4,37.833333,41.40,44.49
5,2019-03-28,Thu,2.0,DET,Away,L,115-98,33,5-15,33.3,...,4,0,1,3,4,12,31.4,44.433333,45.10,46.81
6,2019-03-26,Tue,1.0,MIA,Away,W,104-99,34,10-18,55.6,...,5,1,1,2,2,24,54.7,49.400000,51.32,46.71
7,2019-03-25,Mon,3.0,PHI,Home,W,119-98,29,11-21,52.4,...,4,0,0,0,0,28,47.2,46.466667,48.42,47.40
8,2019-03-22,Fri,2.0,MEM,Home,W,123-119 OT,41,12-23,52.2,...,3,2,2,2,0,25,46.3,51.566667,47.54,47.69
9,2019-03-20,Wed,3.0,NO,Home,W,119-96,26,5-14,35.7,...,3,3,0,1,3,15,45.9,49.533333,47.58,47.12
